In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32


In [6]:
DIRECTORY =r"E:\Data science\Semester 2\Image processing\Face-Mask-Detection-master\Face-Mask-Detection-master\dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [7]:
print("[INFO] loading images...")

data = []
labels = []


[INFO] loading images...


In [8]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)

C:\Users\NITRO\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [9]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)


In [10]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)


In [11]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [12]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

In [13]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


In [14]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [15]:
for layer in baseModel.layers:
	layer.trainable = False


In [16]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

INIT_LR = 1e-4  # Initial learning rate
EPOCHS = 20     # Number of epochs
BS = 32         # Batch size

print("[INFO] compiling model...")
lr_schedule = ExponentialDecay(
    initial_learning_rate=INIT_LR,
    decay_steps=1000,  # Adjust as needed
    decay_rate=0.9    # Adjust as needed
)
opt = Adam(learning_rate=lr_schedule)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
print('Compile done.')


[INFO] compiling model...
Compile done.


In [ ]:
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)


[INFO] training head...
Epoch 1/20


95/95 [==============================] - 100s 995ms/step - loss: 0.3675 - accuracy: 0.8757 - val_loss: 0.1358 - val_accuracy: 0.9817
Epoch 2/20
95/95 [==============================] - 93s 981ms/step - loss: 0.1340 - accuracy: 0.9661 - val_loss: 0.0724 - val_accuracy: 0.9896
Epoch 3/20
95/95 [==============================] - 90s 951ms/step - loss: 0.0903 - accuracy: 0.9759 - val_loss: 0.0529 - val_accuracy: 0.9896
Epoch 4/20
95/95 [==============================] - 92s 967ms/step - loss: 0.0724 - accuracy: 0.9789 - val_loss: 0.0455 - val_accuracy: 0.9896
Epoch 5/20
95/95 [==============================] - 88s 926ms/step - loss: 0.0701 - accuracy: 0.9799 - val_loss: 0.0416 - val_accuracy: 0.9909
Epoch 6/20
78/95 [=======================>......] - ETA: 46:27 - loss: 0.0569 - accuracy: 0.9823

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:

predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))


In [ ]:
print("[INFO] saving mask detector model...")
model.save("mask_detector.h5")

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
import tensorflow as tf

# Assuming your model is named 'maskNet'
model.save("mask_detector.keras", save_format="keras")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming H.history is defined and contains data for "loss", "val_loss", "accuracy", and "val_accuracy"
# Assuming N is defined as the number of epochs

# Generate the x-axis values using np.arange(0, N)
x_values = np.arange(0, N)

# Extract the y-axis values from H.history
train_loss = H.history["loss"]
val_loss = H.history["val_loss"]
train_acc = H.history["accuracy"]
val_acc = H.history["val_accuracy"]

# Print the lengths of the arrays for debugging
print("Length of x_values:", len(x_values))
print("Length of train_loss:", len(train_loss))
print("Length of val_loss:", len(val_loss))
print("Length of train_acc:", len(train_acc))
print("Length of val_acc:", len(val_acc))

# Check if the lengths of y-axis values are consistent with x-axis values
assert len(x_values) == len(train_loss) == len(val_loss) == len(train_acc) == len(val_acc), "Data lengths are not consistent."

# Plot the data
plt.figure()
plt.plot(x_values, train_loss, label="train_loss")
plt.plot(x_values, val_loss, label="val_loss")
plt.plot(x_values, train_acc, label="train_acc")
plt.plot(x_values, val_acc, label="val_acc")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss / Accuracy")
plt.title("Training and Validation Metrics")
plt.show()
